In [1]:
import matplotlib.pyplot as plt
import numpy as np
import sympy
from src.v2.impl.conditions import StepCountCondition, PrecisionCondition, AbsolutePrecisionCondition
from src.v2.impl.methods import CoordinateDescent, GoldenRatioMethod
from src.v2.impl.metrics import StepCount, CallCount, GradientCallCount, HessianCallCount, PrecisionCount, \
    AbsolutePrecisionCount, AbsolutePrecision, MinAbsolutePrecision
from src.v2.impl.oraculs import LambdaOracul, SymbolOracul
from src.v2.runner.debug import FULL_DEBUG
from src.v2.runner.runner import Runner, FULL_VISUALIZE, NO_VISUALIZE
from src.v2.visualization.animation import Animator, FULL_ANIMATION
from src.v2.runner.runner import TABLE
from src.v2.impl.methods import GradientDescent, ScipyMethod, Newton, NewtonBase
from IPython.display import display, HTML

In [2]:
diff(symbols('x y'), symbols('x')) == diff(symbols('y x'), symbols('x'))

False

In [3]:
x, y, z = symbols('x y z')
expr = x**3 + 4*x*y - z
type(expr)

sympy.core.add.Add

In [4]:
from src.common import Point
from src.common.oracul import SymbolOracul, GradientLambdaOracul

x, y, z  = symbols('x y z')
expr = x * 0.26 * (x ** 2 + y ** 2) - 0.48 * x * y 
test = SymbolOracul(expr, ['x', 'y'])
base_differential = GradientLambdaOracul(lambda x, y: 0.26 * (x ** 2 + y ** 2) - 0.48 * x * y,
                                         lambda x, y: np.array([0.52 * x - 0.48 * y, 0.52 * y - 0.48 * x],
                                                               dtype=np.float64))

In [5]:
test.evaluate_hessian(Point(np.array([1, 2, 3])))

array([[1.56, 0.56],
       [0.56, 0.52]])

In [2]:
from src.v2.impl.methods import SteepestDescent
#self.c1 = params["c1"]
#        self.c2 = params["c2"]
 #       self.eps = params["aprox_dec"]
  #      self.max_iters = params["max_iters"]
wolfe = SteepestDescent(c1=0.1, c2=0.2, aprox_dec=1e-11, max_iters=20)

In [3]:

from src.v2.impl.oraculs import LambdaOracul
#bad_defined_absolute = SymbolOracul(1000 * (x - 100) ** 10 + 100 * (y + 20) ** 10, ['x', 'y'])
bad_defined = LambdaOracul(lambda x, y: 1000 * (np.float64(x) - 100) ** 10 + 100 * (np.float64(y) + 20) ** 10)
min_point = np.array([100, -20])
metrics = [StepCount(), CallCount(), GradientCallCount(), HessianCallCount(),
           AbsolutePrecisionCount(0.001, min_point),
           MinAbsolutePrecision(min_point)]
conditions = [StepCountCondition(1000), PrecisionCondition(0.00001)]
#AbsolutePrecisionCondition(0.0001, min_point)            
animations = [Animator()]

modules = animations + metrics + conditions
methods2 = [
    Newton(method=wolfe, aprox_dec=1e-14)
]
oraculs = [bad_defined]
point = np.array([-100, -200])
result = Runner.run(methods2, oraculs, point, modules, precision=1e-5, **TABLE, **NO_VISUALIZE)


LambdaOracul
+----------------------------------------------+-------------+-------------+---------------------+--------------------+---------------------------------+------------------------+
| Method name                                  |   StepCount |   CallCount |   GradientCallCount |   HessianCallCount |   AbsolutePrecisionCount(0.001) |   MinAbsolutePrecision |
+==============================================+=============+=============+=====================+====================+=================================+========================+
| Newton(3,Wolfe(0.1,0.2,eps=1e-11),eps=1e-14) |           2 |         791 |                   2 |                  2 |                               1 |            6.51092e-10 |
+----------------------------------------------+-------------+-------------+---------------------+--------------------+---------------------------------+------------------------+
